In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score,
    roc_auc_score, average_precision_score,
    classification_report
)
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import xgboost as xgb

In [2]:
train_df = pd.read_csv('../../dataset/closed_world/closedworld_train.csv')
test_df  = pd.read_csv('../../dataset/closed_world/closedworld_test.csv')

# feature/target 분리
X = train_df.drop(columns=["label"]).values
y = train_df["label"].values

X_test = test_df.drop(columns=["label"]).values
y_test = test_df["label"].values

In [3]:
# Base model : MLP
mlp_model = MLPClassifier(
    hidden_layer_sizes=(128, 320, 448),
    activation='tanh',
    solver='adam',
    learning_rate_init=0.0003541348658851275,
    alpha=3.154728434953602e-05,
    batch_size=256,
    max_iter=300,
    random_state=42
)

In [4]:
# Base model : XGBoost
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=95,  # label range: 0~94

    # === Optuna Best Params ===
    learning_rate=0.19721925924769965,
    max_depth=10,
    min_child_weight=4,
    subsample=0.7577072390567685,
    colsample_bytree=0.6494394276320198,
    gamma=0.06948732401595321,
    reg_lambda=3.2137571796403774,
    reg_alpha=0.446034681615163,

    n_estimators=300,          # boosting rounds
    eval_metric="mlogloss",
    tree_method="hist",
    random_state=42
)

In [5]:
# Base model : SVM
svm_model = SVC(
    C=33.370899078502916,
    kernel='rbf',
    gamma=0.055876612422341064,
    probability=True,
    random_state=42
)

In [6]:
# Generate Out-of-Fold Predictions (OOF)
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

n_classes = 95
oof_mlp = np.zeros((len(X), n_classes))
oof_xgb = np.zeros((len(X), n_classes))
oof_svm = np.zeros((len(X), n_classes))

print("Generating OOF predictions...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"[Fold {fold+1}/{K}]")

    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]

    # --- MLP ---
    mlp_model.fit(X_tr, y_tr)
    oof_mlp[val_idx] = mlp_model.predict_proba(X_val)

    # --- XGBoost ---
    xgb_model.fit(X_tr, y_tr)
    oof_xgb[val_idx] = xgb_model.predict_proba(X_val)

    # --- SVM ---
    svm_model.fit(X_tr, y_tr)
    oof_svm[val_idx] = svm_model.predict_proba(X_val)

# Meta Training Set (OOF)
train_meta = np.hstack([oof_mlp, oof_xgb, oof_svm])  # shape: (N, 95*3 = 285)
print("Meta feature shape:", train_meta.shape)

# Meta Model (Logistic Regression)
meta_model = LogisticRegression(
    multi_class='multinomial',
    max_iter=500,
    n_jobs=-1
)

print("\nTraining meta model...")
meta_model.fit(train_meta, y)

Generating OOF predictions...
[Fold 1/5]


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[Fold 2/5]


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[Fold 3/5]


...

/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[Fold 5/5]


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Meta feature shape: (13300, 285)

Training meta model...


LogisticRegression(max_iter=500, multi_class='multinomial', n_jobs=-1)

In [7]:
# Test-time predictions
print("\nTraining base models on FULL TRAIN SET...")

mlp_model.fit(X, y)
xgb_model.fit(X, y)
svm_model.fit(X, y)

print("Computing test meta features...")

proba_mlp_test = mlp_model.predict_proba(X_test)
proba_xgb_test = xgb_model.predict_proba(X_test)
proba_svm_test = svm_model.predict_proba(X_test)

test_meta = np.hstack([proba_mlp_test, proba_xgb_test, proba_svm_test])

stack_proba = meta_model.predict_proba(test_meta)
stack_pred = np.argmax(stack_proba, axis=1)

# Evaluation
print("\n========= FINAL STACKING EVALUATION ==========\n")

acc = accuracy_score(y_test, stack_pred)
f1_macro = f1_score(y_test, stack_pred, average='macro')
f1_micro = f1_score(y_test, stack_pred, average='micro')
f1_weighted = f1_score(y_test, stack_pred, average='weighted')

roc_auc_macro = roc_auc_score(
    y_test, stack_proba,
    multi_class='ovr',
    average='macro'
)

pr_auc_macro = average_precision_score(
    y_test, stack_proba,
    average='macro'
)

print(f"Accuracy            : {acc:.6f}")
print(f"F1 (macro)          : {f1_macro:.6f}")
print(f"F1 (micro)          : {f1_micro:.6f}")
print(f"F1 (weighted)       : {f1_weighted:.6f}")
print(f"ROC-AUC (macro)     : {roc_auc_macro:.6f}")
print(f"PR-AUC (macro)      : {pr_auc_macro:.6f}")

print("\nClassification Report:")
print(classification_report(y_test, stack_pred, digits=4))


Training base models on FULL TRAIN SET...
Computing test meta features...

========= FINAL STACKING EVALUATION ==========

Accuracy            : 0.798772
F1 (macro)          : 0.798304
F1 (micro)          : 0.798772
F1 (weighted)       : 0.798304
ROC-AUC (macro)     : 0.986043
PR-AUC (macro)      : 0.853859

Classification Report:
              precision    recall  f1-score   support

           0     0.8182    0.7500    0.7826        60
           1     0.8852    0.9000    0.8926        60
           2     0.9000    0.9000    0.9000        60
           3     0.7903    0.8167    0.8033        60
           4     0.9375    0.7500    0.8333        60
           5     0.8776    0.7167    0.7890        60
           6     0.8182    0.9000    0.8571        60
           7     0.8421    0.8000    0.8205        60
           8     0.7419    0.7667    0.7541        60
           9     0.7018    0.6667    0.6838        60
          10     0.8478    0.6500    0.7358        60
          11     